In [2]:
import pandas as pd
from connect_to_api2 import ResRobot
import requests
import streamlit as st
from dotenv import load_dotenv
import os
from datetime import datetime

# Load environment variables
load_dotenv()

API_KEY = os.getenv("API_KEY")

timetable = ResRobot()

In [3]:
location_name = "Angered centrum"
# Get the first extId
ext_id = timetable.get_location_ext_id(location_name)

if ext_id:
    ext_id = ext_id[0]["extId"]  # Extract only the first extId
    print(f"\nThe first extId is: {ext_id}")
else:
    print("\nNo valid extId found.")



The first extId is: 740025606


In [4]:
# Function to fetch timetable data
def fetch_timetable(ext_id):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={ext_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if "Departure" not in data:
            st.error("No departure data available for this station.")
            return pd.DataFrame()

        # Convert departure data to DataFrame
        df = pd.DataFrame(data["Departure"])
        return df
    except requests.exceptions.RequestException as err:
        st.error(f"Network or HTTP error: {err}")
        return pd.DataFrame()

In [5]:
df_ = fetch_timetable(ext_id)
df_.head(5)

,JourneyDetailRef,JourneyStatus,ProductAtStop,Product,Notes,name,type,stop,stopid,stopExtId,lon,lat,time,date,reachable,direction,directionFlag
0,{'ref': '1|27420|3|1|26012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 74,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,22:30:00,2025-01-26,True,Partille centrum,1
1,{'ref': '1|151954|6|1|26012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 71,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,22:32:00,2025-01-26,True,Eriksbo Östergärde (Göteborg kn),2
2,{'ref': '1|124987|36|1|26012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 8E,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,22:34:00,2025-01-26,True,Gamlestads Torg Västra (Göteborg kn),2
3,{'ref': '1|116794|1|1|26012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 401,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,22:35:00,2025-01-26,True,Kungälv resecentrum,2
4,{'ref': '1|72827|30|1|26012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss 4E,ST,Angered centrum (Göteborg kn),A=1@O=Angered centrum (Göteborg kn)@X=12049213...,740025606,12.049213,57.796073,22:37:00,2025-01-26,True,Gamlestads Torg Västra (Göteborg kn),2


In [6]:
# Function to calculate minutes remaining for departures
def calculate_minutes_remaining(df):
    if df.empty:
        return df

    current_time = datetime.now()
    df['departure_datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df['minutes_remaining'] = (df['departure_datetime'] - current_time).dt.total_seconds() // 60
    
    df = df[df['minutes_remaining'] >= 0]
    
    # Make it User friendly the interface
    df['name'] = df['name'].str.replace(r'^Länstrafik - ', '', regex=True)
    df['stop'] = df['stop'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)
    df['direction'] = df['direction'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)

    return df[['name', 'stop', 'direction', 'minutes_remaining']]


In [9]:
df = fetch_timetable(ext_id)  # Run this first
df__ = calculate_minutes_remaining(df)  # Then run this
df__.head(6)


/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_3374/2819433555.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].str.replace(r'^Länstrafik - ', '', regex=True)
/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_3374/2819433555.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stop'] = df['stop'].str.replace(r'\s*\(Göteborg kn\)', '', regex=True)
/var/folders/5w/rtw5hcjn25q4yp0062_8_m8h0000gn/T/ipykernel_3374/2819433555.py:15: SettingWithCopyWarning: 
A v

,name,stop,direction,minutes_remaining
1,Buss 71,Angered centrum,Eriksbo Östergärde,1.0
2,Buss 8E,Angered centrum,Gamlestads Torg Västra,3.0
3,Buss 401,Angered centrum,Kungälv resecentrum,4.0
4,Buss 4E,Angered centrum,Gamlestads Torg Västra,6.0
5,Buss 8E,Angered centrum,Gamlestads Torg Västra,11.0
6,Buss 73,Angered centrum,Peppargatan,14.0
